In [163]:
import rds_config
from sqlalchemy import create_engine

#rds settings
db_host  = rds_config.db_host
db_username = rds_config.db_username
db_password = rds_config.db_password
db_name = rds_config.db_name


SQLALCHEMY_DATABASE_URI = 'mysql+pymysql://' + db_username + ':' + db_password + '@' + db_host + '/' + db_name + '?charset=utf8'

engine = create_engine(SQLALCHEMY_DATABASE_URI)
connection = engine.connect()

In [164]:
import pandas as pd
sql = "SELECT oid, aid, substring(date, 6,2) as month, substring(date, 9,2) as date, media, title, org_url, org_count+naver_count as total_count, section, ml_section FROM newscrawl WHERE section!='스포츠ㆍ연예' and ( org_count>0 or naver_count>0) "
df = pd.read_sql(sql, connection)
#pd.read_csv("dcdata10.csv")

In [165]:
len(df)

454360

In [166]:
df['title']=df['title'].str.replace(r'\[[A-Za-z가-힣0-9 ]+\]|\([A-Za-z가-힣0-9 ]+\)|\<[A-Za-z가-힣0-9 ]+\>','')
df['title']=df['title'].str.strip()
df['file_name']=pd.to_numeric(df["aid"], downcast='integer')

In [167]:
df['thumbnail']="http://imgnews.naver.net/image/origin/"+df["oid"]+"/2016/"+df["month"]+"/"+df["date"]+"/"+df["file_name"].apply(str)+".jpg"

In [168]:
top100=df[['total_count', 'title', 'org_url', 'thumbnail']].sort_values('total_count', ascending=False).head(100)
top100.to_csv('data/top100.tsv', sep="\t", index=False)

In [169]:
%%time
from konlpy.tag import Mecab
mecab = Mecab()

def mecab_words(sentence):
    word_list = mecab.nouns(sentence)
    #return " ".join(word_list)
    return " ".join(filter(lambda x: len(x.strip())>1, word_list))
    #return filter(lambda x: len(x)>1, " ".join(word_list))

df['title_me_noun'] = df['title'].apply(mecab_words)

CPU times: user 34.6 s, sys: 76 ms, total: 34.7 s
Wall time: 34.7 s


In [221]:
word_count=df[['ml_section', 'month', 'total_count', 'media', 'title_me_noun']]
#word_count=word_count[word_count['month']=='11']

In [222]:
df_by_count = word_count.groupby(['total_count'])
count_list = df_by_count['title_me_noun'].apply(list)

In [223]:
result_by_count = pd.Series()
word_dict = dict()
for weight, word_list in count_list.iteritems():
#    result_by_count.set_value(weight, Counter(" ".join(word_list).split(" ")))
    #print(" ".join(word_list))
    for corpus in word_list :
        for word in corpus.split(" "):
            word_dict[word] = word_dict.get(word, 0)+weight

In [224]:
from collections import OrderedDict
import itertools
d_sorted_by_value = OrderedDict(sorted(word_dict.items(), key=lambda x: x[1], reverse=True))
top100 = itertools.islice(d_sorted_by_value.items(), 0, 100)
for k,v in top100:
    print("%s:%d"%(k,v))

대통령:9250363
최순실:4891858
박근혜:3695649
세월호:2300667
한국:2202757
경찰:1953772
논란:1946919
청와대:1880217
탄핵:1815637
검찰:1772960
정부:1601732
여성:1539458
의원:1507701
시간:1474976
국민:1458910
의혹:1442353
사람:1408395
수사:1394549
촛불:1385455
시민:1352681
이유:1273842
새누리:1264243
공개:1236638
우병우:1183370
서울:1125650
국회:1050701
사드:1026643
일본:1009027
교수:1000710
위안부:998166
퇴진:993481
집회:985423
사망:984393
학생:975998
우리:972460
정유라:872002
국정:871374
문재인:869890
세계:862920
조사:858360
발언:853719
이정현:828756
사진:828267
이재명:818269
정치:814946
시장:801090
더민주:766015
대표:765244
소녀:762706
시위:760296
백남기:743979
촛불집회:743685
새누리당:732142
중국:727891
아이:715956
박원순:713709
교과서:706318
청년:702863
반대:701514
기업:700347
박정희:700158
발견:687533
가능:685810
나라:669081
부산:658620
아들:653509
한겨레:641136
삼성:639863
사건:628317
요구:626582
사과:626430
미국:618710
거부:614188
사고:613717
특검:606158
병원:602361
문제:602100
엄마:593221
노무현:588172
성폭행:585155
남성:582779
학교:582744
눈물:581871
북한:573932
트럼프:569329
반기문:567845
단체:562014
역사:560352
국가:552048
지시:551477
지진:550010
대선:549880
분노:546107
운동:539209
인사:53

## 섹션별 또는 월별 단어 빈도 * 공유수

In [225]:
df_by_month_count = word_count.groupby(['ml_section','total_count'])
month_count_list = df_by_month_count['title_me_noun'].apply(list)
month_count_list.shape

(17675,)

In [226]:
result_by_month = pd.Series()
i_count = 0
word_dict = dict()
prior_month = '01'
for (month, weight), word_list in month_count_list.iteritems():
    if(prior_month!=month):
        word_dict = dict()        
    for corpus in word_list :
        for word in corpus.split(" "):
            word_dict[word] = word_dict.get(word, 0)+weight  
            
    result_by_month.set_value(month, word_dict)
    prior_month=month
            


In [227]:
for month, words in result_by_month.iteritems():
    d = OrderedDict(sorted(words.items(), key=lambda x: x[1], reverse=True))
    x = itertools.islice(d.items(), 0, 30)
    print(month,[k for k,v in x]  )

0 ['대통령', '한겨레', '한국', '박근혜', '최순실', '서울', '이유', '여성', '교과서', '정부', '국정', '사드', '사람', '세월호', '우리', '논란', '세계', '시민', '', '나라', '시장', '시간', '교사', '중국', '그림판', '국민', '역사', '학생', '위안부', '학교']
1 ['대통령', '최순실', '박근혜', '탄핵', '새누리', '의원', '청와대', '국회', '문재인', '세월호', '더민주', '이정현', '우병우', '국민', '검찰', '정부', '새누리당', '논란', '정치', '시간', '의혹', '사드', '이재명', '대표', '김무성', '발언', '필리버스터', '안철수', '지지율', '촛불']
2 ['한국', '기업', '삼성', '정부', '경제', '최순실', '삼성전자', '회장', '에어컨', '전기료', '세계', '시장', '직원', '은행', '판매', '국민연금', '시간', '논란', '위기', '국민', '요금', '배터리', '갤노트', '의혹', '확인', '카드', '최대', '중국', '이유', '투자']
3 ['대통령', '최순실', '경찰', '박근혜', '세월호', '검찰', '수사', '논란', '시민', '촛불', '청와대', '여성', '의혹', '집회', '학생', '서울', '교수', '사람', '시간', '한국', '사망', '정유라', '정부', '백남기', '촛불집회', '국민', '공개', '퇴진', '우병우', '이유']
4 ['한국', '논란', '사진', '영화', '블랙리스트', '이유', '세계', '사람', '박근혜', '예술', '등장', '대통령', '문화', '의정부', '우리', '무대', '귀향', '공개', '홍상수', '교수', '여자', '청와대', '조윤선', '올림픽', '작가', '위안부', '최순실', '이야기', '화제', '아이']
5 ['한국', '여성', '트럼프', '일본', 

## 월별 단어 빈도

In [218]:
df_by_month = df.groupby(['month'])
df_by_media_month = df.groupby(['month', 'media'])

In [219]:
from collections import Counter

by_me_month = df_by_month['title_me_noun'].apply(list)
result_by_me_month = pd.Series()

for month, words in by_me_month.iteritems():
    result_by_me_month.set_value(month, Counter(" ".join(words).split(" ")))

In [220]:
for month, words in result_by_me_month.iteritems():
    print(month, words.most_common(30))

01 [('한국', 1033), ('대통령', 892), ('중국', 786), ('서울', 665), ('위안부', 640), ('경제', 632), ('더민주', 612), ('정부', 606), ('시장', 576), ('북한', 560), ('세계', 548), ('논란', 527), ('안철수', 507), ('미국', 496), ('의원', 437), ('한파', 421), ('가능', 420), ('일본', 415), ('정치', 406), ('청년', 393), ('기업', 385), ('합의', 379), ('공항', 378), ('핵실험', 373), ('영입', 370), ('올해', 366), ('경찰', 363), ('사망', 363), ('대북', 358), ('제주', 351)]
02 [('한국', 872), ('개성공단', 746), ('대통령', 723), ('중국', 666), ('사드', 655), ('정부', 642), ('미사일', 642), ('북한', 638), ('서울', 543), ('기업', 517), ('더민주', 508), ('시장', 502), ('세계', 472), ('가능', 466), ('제재', 458), ('경제', 441), ('발사', 440), ('필리버스터', 432), ('의원', 428), ('공천', 412), ('대북', 412), ('배치', 410), ('논란', 405), ('총선', 404), ('미국', 368), ('국민의당', 356), ('시간', 341), ('경찰', 336), ('국회', 331), ('공개', 326)]
03 [('공천', 1335), ('더민주', 971), ('한국', 914), ('총선', 849), ('김종인', 816), ('후보', 755), ('대통령', 752), ('서울', 673), ('대표', 670), ('유승민', 589), ('국민의당', 553), ('김무성', 552), ('이세돌', 549), ('여성', 529), (

In [22]:
df_section = df[['section', 'ml_section', 'media', 'total_count']]
df_section[['media','ml_section','total_count']].groupby(['ml_section', 'media']).agg(['count','sum'])

total_count         
                          count      sum
ml_section media                        
0          chosun         20061  1775041
           donga          19648   885702
           edaily          3216   121187
           hani            4887  2030009
           joins           6200   836716
           jtbc             503    61297
           kbs             1498    93530
           kh              8799  1319687
           mbc             3138    84109
           mk              6234   288059
           nocut           4896   272554
           ohmynews        5600  1573652
           sbs             3425   538344
           yonhap         44774  3107181
           ytn             3702   942635
1          chosun          8204  1537788
           donga            620    83773
           edaily          3334   218287
           hani            5160  4495729
           joins           6773  3140421
           jtbc            4406  2148632
           kbs             4741   241284
           kh              6304  3276232
           mbc             1839    74180
           mk              2946   193754
           nocut           4174   633833
           ohmynews        5629  5802872
           sbs             8057  1278525
           yonhap         15671  1800274
           ytn             3323   406840
...                         ...      ...
4          kbs             1665    81617
           kh              1494   281243
           mk              1693    72439
           nocut           1978   245889
           ohmynews         563   116121
           sbs              602   112647
           yonhap          1761   131566
           ytn             2489   689706
5          chosun          5189  1678389
           donga             93     4150
           edaily           931    15893
           hani            2248   600828
           joins           3676   843361
           jtbc            1342   208850
           kbs             3190   103161
           kh              2709   762311
           mk              2657    98782
           ohmynews        1512   780875
           sbs             9382  1713128
           yonhap         12190   595519
           ytn             3650  1684294
6          edaily          2035    82683
           hani             316    84669
           joins            255    42577
           kbs              475    25725
           kh               222    10488
           mk              1664    82107
           nocut            406    57162
           yonhap          2012   100289
           ytn              242    97989

[95 rows x 2 columns]